## Load libraries

https://www.tensorflow.org/lite/guide/python

In [1]:
import cloudpickle as pickle
import re
import numpy as np

In [21]:
!pip install https://github.com/google-coral/pycoral/releases/download/release-frogfish/tflite_runtime-2.5.0-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 1.2MB 449kB/s eta 0:00:01


In [2]:
from tflite_runtime.interpreter import Interpreter

In [16]:
!pip install keras_preprocessing

In [3]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing import sequence

In [13]:
!pip install wn

     |████████████████████████████████| 31.6MB 7.9MB/s eta 0:00:011
  Stored in directory: /home/myself/.cache/pip/wheels/56/e3/c4/886021dbf4d758dc3cb9ddaa47d7d6fc895240d83f010e6305
Successfully built wn


In [4]:
'''
Using standalone wordnet instead of nltk:

def lemmatize(self, word, pos=NOUN):
    lemmas = wordnet._morphy(word, pos)
    return min(lemmas, key=len) if lemmas else word
'''
from wn.morphy import _morphy

## Load model & dependencies

In [5]:
path = "resources/"

In [6]:
# Load model
interpreter = Interpreter(
    model_path=path + 'fcc_sms_classification.tflite'
)
interpreter.allocate_tensors()

In [20]:
class objectview(object):
    """
    Trick to access dictionary items as object attributes
    """
    def __init__(self, d):
        self.__dict__ = d

with open(path + "utils.pkl", "rb") as f:
    utils = objectview(pickle.load(f))

## Preprocesssing txt

In [9]:
def lemmatize(word, pos='n'):
    '''
    Parts of speech constants:
    ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
    '''
    lemmas = _morphy(word, pos)
    return min(lemmas, key=len) if lemmas else word

lemmatize('abaci')

# https://pypi.org/pypi/wn/json
# print(inspect.getsource(WordNetLemmatizer))

'abaci'

In [10]:
def cleanup(txt):
    txt = re.sub(r'([^\s\w])+', ' ', txt)
    txt = " ".join([lemmatize(word) for word in txt.split()
                    if not word in utils.stopwords_eng])
    txt = txt.lower()
    return txt

In [11]:
txt = 'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'
cleanup(txt)

'ahhhh woken bad dream u tho dont like u right didnt know anything comedy night guess im'

In [12]:
utils.texts_to_sequences([cleanup(txt)])

[[309, 227, 1, 587, 42, 15, 1, 90, 359, 13, 103, 54, 228, 86]]

In [13]:
max_len = 500

def preprocessing(X):
    return utils.pad_sequences(
        utils.texts_to_sequences([cleanup(x) for x in X]),
        maxlen=max_len)

preprocessing([txt])[0][-5:]

array([ 13, 103,  54, 228,  86], dtype=int32)

## Predict

https://medium.com/@mmohamedrashik/how-to-deploy-tensorflow-regression-model-in-android-tf-lite-part-2-90b9ebb31903

In [14]:
def predict(X):
    input_index  = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    input_data = np.array(X, dtype=np.float32)
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()

    return interpreter.get_tensor(output_index)

In [15]:
predict(preprocessing([
    "you have won £1000 cash! call to claim your prize."
]))

array([[1.3542966]], dtype=float32)

## Set FastAPI settings

In [15]:
!pip install fastapi

     |████████████████████████████████| 51kB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 6.5MB/s eta 0:00:011
     |████████████████████████████████| 112kB 8.4MB/s eta 0:00:01


In [19]:
!pip install uvicorn

     |████████████████████████████████| 51kB 2.6MB/s eta 0:00:011


In [16]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

In [17]:
# Using Pydantic BaseModel class for automatic data validation
class Data(BaseModel):
    text: str

    class Config:
        schema_extra = {
            "example": {
                "text": "you have won £1000 cash! call to claim your prize."
            }
        }

# Defining Response format for documentation
class Response(BaseModel):
    p: float
    prediction: str

    class Config:
        schema_extra = {
            "example": {
                "prediction": "ham",
                "p": 0
            }
        }

In [18]:
app = FastAPI(debug=True)

@app.post("/predict", response_model=Response)
def app_predict(data: Data):
    try:
        text = data.text
        res  = predict(preprocessing([text]))[0]
        p    = float(res[0])

        return {"prediction": ("ham" if p<0.5 else "spam"), "p": p}
    except Exception as e:
        print(e)
        return {"prediction" : "error"}
    
@app.get('/healthcheck', status_code=200)
async def healthcheck():
    return 'OK'

* Create a .py file

    ```
    ipython nbconvert FastAPI.ipynb --to script
    ```

* Edit the .py file to remove unnecessary steps — ie function checking, installs

* Start the server

    ```
    uvicorn FastAPI:app --host=127.0.0.1 --port=${PORT:-8000}
    ```

* Go to http://127.0.0.1:8000/docs

## Alternative way to Start server

In [19]:
from os import environ

HOST = environ.get('HOST', "127.0.0.1")
PORT = int(environ.get('PORT', 8000))

print(HOST, ':', PORT)

127.0.0.1 : 8000


In [13]:
if __name__ == '__main__':
    uvicorn.run(app, host=HOST, port=PORT)

* Start the server

    ```
    python FastAPI.py
    ```

## Deploy on Heroku

* Create `runtime.txt` to specify the Python version to use. Make sure it is a [supported runtime](https://devcenter.heroku.com/articles/python-support#supported-runtimes)

    ```
    python-3.7.9
    ```

* Create `Procfile`

    ```
    web: uvicorn FastAPI:app --host=0.0.0.0 --port=${PORT:-5000}
    ```

* Create `requirements.txt`

    ```
    https://github.com/google-coral/pycoral/releases/download/release-frogfish/tflite_runtime-2.5.0-cp37-cp37m-linux_x86_64.whl
    fastapi
    uvicorn
    pydantic
    cloudpickle
    keras_preprocessing
    wn
    ```

* Commit your files.

    ```
    git init
    git add runtime.txt Procfile requirements.txt FastAPI.py models/
    git commit -m "Initial Deployment"
    ```

* Create a Heroku app

    ```
    heroku login
    heroku create
    git push heroku master
    ```

If you want: [Set up auto-deploy](https://towardsdatascience.com/autodeploy-fastapi-app-to-heroku-via-git-in-these-5-easy-steps-8c7958ef5d41)

## Query snippet

In [14]:
"""
import requests

url = 'http://127.0.0.1:8000/predict'

r = requests.post(url, json={
    'text': 'you have won £1000 cash! call to claim your prize.'
})
r.json()
"""

"\nimport requests\n\nurl = 'http://127.0.0.1:8000/predict'\n\nr = requests.post(url, json={\n    'text': 'you have won £1000 cash! call to claim your prize.'\n})\nr.json()\n"